In [17]:
from dataclasses import dataclass
from pathlib import  Path

@dataclass(frozen= True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path


In [18]:
from text_summarizer.constants import *
from text_summarizer.utils.comon import read_yaml, create_directories
import os

class ConfigurationManager:
    def __init__(
            self,
            config_file_path = PROJECT_DIR.joinpath(CONFIG_FILE_PATH),
            params_file_path = PROJECT_DIR.joinpath(PARAMS_FILE_PATH)):
        
        # print( read_yaml(config_file_path))
        # print( read_yaml(os.path("../config/config.yaml")))
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([PROJECT_DIR.joinpath(self.config.artifacts_root)])

    def get_model_evaluation_config(self):
        config = self.config.model_evaluation
        # print(self.config.model_evaluation)
        create_directories([PROJECT_DIR.joinpath(config.root_dir)])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir = PROJECT_DIR.joinpath(config.root_dir),
            data_path = PROJECT_DIR.joinpath(config.data_path),
            model_path = PROJECT_DIR.joinpath(config.model_path),
            tokenizer_path = PROJECT_DIR.joinpath(config.tokenizer_path),
            metric_file_name = PROJECT_DIR.joinpath(config.metric_file_name),
        )
        return model_evaluation_config


In [19]:
from tqdm import tqdm
import pandas as pd
import torch
from datasets import load_dataset, load_from_disk, load_metric
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [20]:
class ModelEvaluation:
    def __init__(self,config: ModelEvaluationConfig):
        self.config = config
    
    def generate_batch_sized_chunks(self,list_of_elements,batch_size):
        """split the dataset into smaller batches that can be processed simultaneously
        Yield successive batch-sized chunks from a list of elements."""
        for i in range(0,len(list_of_elements),batch_size):
            yield list_of_elements[i:i+batch_size]
    
    def calculate_metric_on_test_dataset(self,dataset,metric,model,tokenizer,
                                         batch_size=16,device="cuda" if torch.cuda.is_available() else "cpu",
                                         column_text="article",column_summary="highlights"):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text],batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary],batch_size))
        
        for article_batch, target_batch in tqdm(
                zip(article_batches,target_batches), total=len(article_batches)):

            inputs = tokenizer(article_batch,max_length=1024, truncation=True,padding="max_length",
                               return_tensors="pt")

            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                       attention_mask=inputs["attention_mask"].to(device),
                                       length_penalty=0.8, num_beams=8, max_length = 128)

            decoded_summaries = [tokenizer.decode(summary,skip_special_tokens=True,clean_up_tokenization=True)
                                 for summary in summaries]
            decoded_summaries = [(d.replace(""," ")) for d in decoded_summaries ]   

            metric.add_batch(predictions = decoded_summaries, references=target_batch)

        # Compute and return ROUGE score
        score = metric.compute()
        return score
    
    def evaluate(self):
        device="cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)

        dataset_samsum_pt = load_from_disk(self.config.data_path)

        rouge_name = ["rouge1", "rouge2", "rougeL", "rougeLsum" ]

        rouge_metric = load_metric('rouge')
        score = self.calculate_metric_on_test_dataset(
            dataset_samsum_pt['test'][0:10], rouge_metric,model_pegasus,tokenizer,
            batch_size=2,column_text='dialogue', column_summary= 'summary'
        )


        rouge_dict = dict((rn,score[rn].mid.fmeasure  ) for rn in rouge_name)

        df = pd.DataFrame(rouge_dict , index = ['pegasus'])
        df.to_csv(self.config.metric_file_name , index = False)




In [21]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config= model_evaluation_config)
    model_evaluation_config.evaluate()
except Exception  as e:
    raise e

[2023-06-17 04:14:04,127: text_summarizer_logger - INFO:  comon: line 33, yaml file: C:\Users\Mahmo\Desktop\text_summarizer\config\config.yaml loaded successfully]
[2023-06-17 04:14:04,142: text_summarizer_logger - INFO:  comon: line 33, yaml file: C:\Users\Mahmo\Desktop\text_summarizer\params.yaml loaded successfully]
[2023-06-17 04:14:04,146: text_summarizer_logger - INFO:  comon: line 64, created directory: C:\Users\Mahmo\Desktop\text_summarizer\artifacts]
[2023-06-17 04:14:04,149: text_summarizer_logger - INFO:  comon: line 64, created directory: C:\Users\Mahmo\Desktop\text_summarizer\artifacts\model_evaluation]


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 4.00 GiB total capacity; 3.52 GiB already allocated; 0 bytes free; 3.52 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF